In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from configparser import ConfigParser
import os
import pymysql
import spacy
nlp = spacy.load('en')
import re
import phonenumbers

from imwithdata.es_etl.issues_actions import (
    issues,
    actions,
    state_regex,
    city_regex,
    web_url_regex,
    date_include_regex,
    date_exclude_regex,
    leg_name_regex,
    leg_twitter_regex
)

In [2]:
config_file = os.path.join(os.pardir,'config','config.ini')

def get_ini_vals(ini_file, section):
    config = ConfigParser()
    config.read(ini_file)
    return config[section]

mysql_creds = get_ini_vals(config_file, 'mysql')


In [3]:
engine = create_engine("""mysql+pymysql://{user}:{password}@{host}:{port}/{db}""".format(user=mysql_creds['user'],
                                                                                 password=mysql_creds['password'],
                                                                                 host=mysql_creds['host'],
                                                                                 port=mysql_creds['port'],
                                                                                 db=mysql_creds['database']
                                                                                )
                      )
conn = engine.connect()


In [4]:
metadata = sqlalchemy.MetaData(conn)

In [5]:
metadata.tables.keys()

dict_keys([])

In [6]:
final_data = pd.read_csv('/Users/brosskatz/PycharmProjects/rzst/w210_imwithdata/imwithdata/data/static_data/final_data_example.csv')

In [7]:
# final_data.head()
final_data.columns

Index(['Unnamed: 0', 'issue', 'action', 'id', 'es_score', 'total_score',
       'tweet', 'tweet_timestamp', 'query_timestamp', 'tweet_user',
       'tweet_cities', 'tweet_states', 'tweet_urls', 'tweet_phone_numbers',
       'tweet_dates_ref', 'tweet_legislator_names',
       'tweet_legislator_handles'],
      dtype='object')

In [8]:
final_data.drop(final_data.columns[0], axis=1,inplace=True)

In [9]:
final_data.columns

Index(['issue', 'action', 'id', 'es_score', 'total_score', 'tweet',
       'tweet_timestamp', 'query_timestamp', 'tweet_user', 'tweet_cities',
       'tweet_states', 'tweet_urls', 'tweet_phone_numbers', 'tweet_dates_ref',
       'tweet_legislator_names', 'tweet_legislator_handles'],
      dtype='object')

In [10]:
final_data['action'].unique()

array(['charity', 'protest', 'petition', 'gathering', 'boycott',
       'advocate', 'vote', 'townhall'], dtype=object)

In [57]:
# event_mask =( ((final_data['action'] == 'protest') | (final_data['action'] == 'gathering' ) | (final_data['action'] == 'townhall')
#               | (final_data['action'] == 'boycott'))
#             )

In [11]:
actions = final_data.sort('total_score',ascending=[0])

/Users/brosskatz/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [12]:
actions.head()

,issue,action,id,es_score,total_score,tweet,tweet_timestamp,query_timestamp,tweet_user,tweet_cities,tweet_states,tweet_urls,tweet_phone_numbers,tweet_dates_ref,tweet_legislator_names,tweet_legislator_handles
691,immigrants,advocate,AVsY87p6MNAj8foZQ_de,25.934280,32.934280,Take Action: Sign Petition to Confirm you Stil...,2017-03-29T07:24:24.000Z,2017-03-29T21:59:18.900874,clarkdennycjoy1,[],[],['https://t.co/IGZiJsfDVE'],[],[],[],[]
692,immigrants,advocate,AVsa9QvLMNAj8foZRk0f,25.934280,32.934280,Take Action: Sign Petition to Demand Congress ...,2017-03-29T16:45:01.000Z,2017-03-29T21:59:18.900874,RonNelson15,[],[],['https://t.co/2k4QMSJmH1'],[],[],[],[]
586,climate,advocate,AVsaypZnMNAj8foZRgP9,18.827164,25.827164,Join @NRDC &amp; sign petition to defend our e...,2017-03-29T15:58:41.000Z,2017-03-29T21:59:18.900874,KatabreKuika,[],[],['https://t.co/PwbVSJzSyW'],[],[],[],[]
536,womens_right,advocate,AVsbiUtmMNAj8foZRy9y,19.624722,25.624722,Protect Reproductive Health And Rights #sign ...,2017-03-29T19:26:59.000Z,2017-03-29T21:59:18.900874,susan_marie_ll,[],[],['https://t.co/50eRJVFMgX'],[],[],[],[]
690,immigrants,advocate,AVsYHdxXMNAj8foZQ0Cx,26.235420,25.235420,Sign Petition gov of Sanctuary Cities Held Leg...,2017-03-29T03:30:48.000Z,2017-03-29T21:59:18.900874,GuthrieWilson,[],[],['https://t.co/LVlw9L65ZZ'],[],[],[],[]


In [13]:
issue_list = actions['issue'].tolist()
action_list = actions['action'].tolist()
id_list = actions['id'].tolist()

es_score_list = actions['es_score'].tolist()
total_score_list = actions['total_score'].tolist()
tweet_list = actions['tweet'].tolist()
tweet_timestamp_list = actions['tweet_timestamp'].tolist()
query_timestamp_list = actions['query_timestamp'].tolist()
user_list = actions['tweet_user'].tolist()



In [14]:
time_regex = re.compile(r'\d{1,2}(?:(?:am|pm)|(?::\d{1,2})(?:am|pm)?)', re.IGNORECASE)

In [18]:
dates = []
start_times = []
end_times = []
cities = []
states = []
legislators = []
legislator_handles = []
phone_numbers = []
titles = []

for i, tweet in enumerate(tweet_list):
    date = ''
    start_time = ''
    end_time = ''
    city = ''
    state = ''
    legislator = ''
    legislator_handle = ''
    phone_number = ''
    url = ''
    title = ''
    
    doc = nlp(tweet)
    all_dates = [doc.text for doc in doc.ents if doc.label_ == 'DATE']
    date_matches = re.findall(date_include_regex, ' '.join(all_dates))
    ### EXCLUDE SOME DIRTY DATES FROM TWITTER THAT SPACY MISTAKENLY INCLUDES    
    if date_matches:
        date = date_matches[0]
    dates.append(date)
        
    times = re.findall(time_regex,tweet)
    if times:
        start_time = times[0]
    if len(times) > 1:
        end_time = times[1]
    start_times.append(start_time)
    end_times.append(end_time)
    
    tweet_cities = re.findall(city_regex,tweet)
    if tweet_cities:
        tweet_cities = list(set([city.title() for city in tweet_cities]))
        if len(tweet_cities) == 1:
            city = tweet_cities[0]
        else:
            city = tweet_cities
    cities.append(city)
    
    tweet_states = re.findall(state_regex,tweet)
    if tweet_states:
        tweet_states = list(set(tweet_states))
        if len(tweet_states) == 1:
            state = tweet_states[0]
        else:
            state = tweet_states
    states.append(state)
    
    tweet_legislators = re.findall(leg_name_regex,tweet)
    if tweet_legislators:
        if len(tweet_legislators) == 1:
            legislator = tweet_legislators[0]
        else:
            legislator = tweet_legislators
    legislators.append(legislator)
    
    tweet_leg_handles = re.findall(leg_twitter_regex,tweet)
    if tweet_leg_handles:
        if len(tweet_leg_handles) == 1:
            legislator_handle = tweet_leg_handles[0]
        else:
            legislator_handle = tweet_leg_handles
        legislator_handle = tweet_leg_handles
    legislator_handles.append(legislator_handle)
    
    if phonenumbers.PhoneNumberMatcher(tweet, "US"):
        for i,match in enumerate(phonenumbers.PhoneNumberMatcher(tweet, "US")):
            if i == 0:  
                phone_number = phonenumbers.format_number(match.number,
                                                            phonenumbers.PhoneNumberFormat.NATIONAL)
    phone_numbers.append(phone_number)
    
    tweet_urls = re.findall(web_url_regex,tweet)
    title = tweet
    if tweet_urls:
        for urly in tweet_urls:
            title = title.replace(urly,'')
    titles.append(title)
        
#     print (date, start_time, end_time, city, state, tweet )
    
        
                
#                 print((final_data['action'] == 'protest'),, tweet)

In [19]:
print (len(tweet_list))
print (len(dates))
print (len(start_times))
print (len(end_times))
print (len(cities))
print (len(legislators))
print (len(legislator_handles))
print (len(phone_numbers))
print (len(titles))

906
906
906
906
906
906
906
906
906


In [20]:
print(cities[:3000])

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Mobile', '', '', '', '', '', 'Bowie', '', '', '', '', 'Santa Ana', '', '', '', 'Sacramento', '', '', ['Sacramento', 'Los Angeles'], '', '', '', '', '', 'Mobile', '', '', '', 'Mobile', '', '', '', '', '', '', '', '', '', 'Windsor', '', '', 'Cincinnati', '', '', 'El Centro', 'Washington', '', '', '', '', '', '', '', '', 'Glasgow', ['Plano', 'New Braunfels', 'Kerrville', 'Frederick', 'Abilene', 'Garland', 'Mesquite'], 'Flint', '', '', '', '', '', '', '', '', '', '', 'Sacramento', '', '', '', '', '', '', '', 'Sacramento', '', '', '', '', '', '', '', '', '', 'Sacramento', '', '', '', '', '', 'Wayne', '', 'Miami', 'Brockton', 'Flint', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Hollywood', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Seattle', '', '', '', '', '', '', '', '', '', '', 'Niagara Falls', 'Niagara Falls', '', '', '', '', '', '', '', '', '', 'Sacramento', '', '', '', '', '', '', '

In [21]:
prepped_to_sql = pd.DataFrame(
                        {'title': titles,
                         'description': tweet_list,
                         'action': action_list,
                         'issue':issue_list,
                         'total_score':total_score_list,
                         'relevance_score':es_score_list,
                         'legislators': legislators,
                         'legislator_twitter': legislator_handles,
                         'city':cities,
                         'state':states,
                         'phone_number': phone_numbers,
                         'announced_date': tweet_timestamp_list,
                         'query_date':query_timestamp_list
                        })


# * title
# * description
# * action
# * issue
# * total_score
# * relevance_score
# * legislators
# * city
# * state
# * phone_numbers
# * announced_date
# * query_date


In [24]:
prepped_to_sql[:100]

,action,announced_date,city,description,issue,legislator_twitter,legislators,phone_number,query_date,relevance_score,state,title,total_score
0,advocate,2017-03-29T07:24:24.000Z,,Take Action: Sign Petition to Confirm you Stil...,immigrants,,,,2017-03-29T21:59:18.900874,25.934280,,Take Action: Sign Petition to Confirm you Stil...,32.934280
1,advocate,2017-03-29T16:45:01.000Z,,Take Action: Sign Petition to Demand Congress ...,immigrants,,,,2017-03-29T21:59:18.900874,25.934280,,Take Action: Sign Petition to Demand Congress ...,32.934280
2,advocate,2017-03-29T15:58:41.000Z,,Join @NRDC &amp; sign petition to defend our e...,climate,,,,2017-03-29T21:59:18.900874,18.827164,,Join @NRDC &amp; sign petition to defend our e...,25.827164
3,advocate,2017-03-29T19:26:59.000Z,,Protect Reproductive Health And Rights #sign ...,womens_right,,,,2017-03-29T21:59:18.900874,19.624722,,Protect Reproductive Health And Rights #sign ...,25.624722
4,advocate,2017-03-29T03:30:48.000Z,,Sign Petition gov of Sanctuary Cities Held Leg...,immigrants,,,,2017-03-29T21:59:18.900874,26.235420,,Sign Petition gov of Sanctuary Cities Held Leg...,25.235420
5,advocate,2017-03-29T13:39:22.000Z,,Let's go people sign petition for this Union t...,voting,,,,2017-03-29T21:59:18.900874,16.916447,,Let's go people sign petition for this Union t...,22.916447
6,advocate,2017-03-29T22:09:44.000Z,,Call Your Senator: Oppose Trump’s Nominee http...,immigrants,,,,2017-03-29T21:59:18.900874,16.620247,,Call Your Senator: Oppose Trump’s Nominee,22.620247
7,petition,2017-03-29T18:36:43.000Z,,Sign the #IStandWithPP petition and learn how ...,womens_right,,,,2017-03-29T21:59:18.900874,15.091169,,Sign the #IStandWithPP petition and learn how ...,22.091169
8,petition,2017-03-29T20:42:28.000Z,,"Join the call for sustainability fishing, sign...",climate,,,,2017-03-29T21:59:18.900874,14.410313,,"Join the call for sustainability fishing, sign...",21.410313
9,advocate,2017-03-29T08:41:07.000Z,,Please sign petition to free civil rights acti...,civil_rights,,,,2017-03-29T21:59:18.900874,19.314816,,Please sign petition to free civil rights acti...,20.314816
